In [ ]:
import os
import numpy as np
import cv2
import keras
import keras as K
from keras.utils import Sequence
from keras.models import Sequential
from keras.layers import TimeDistributed, Reshape,Permute,MaxPooling2D, LSTM,Dense,Lambda, Activation, BatchNormalization, Bidirectional, GRU, LeakyReLU, Dropout, Conv2D, Flatten, Input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
import random
from keras import optimizers
np.set_printoptions(threshold=np.nan)

In [ ]:
# #resize and make new dir for our train data...

# dir = "./train/NORMAL"
# for root,dirs,files in os.walk(dir):
#     for x,file in enumerate(files):
#         curdir =  os.path.join(root,file)
#         img = cv2.imread(curdir,0) 
#         img = cv2.resize(img, (299,299))
#         cv2.imwrite("./train/NORMAL/resized/%d.jpg" % x, img)
        

In [ ]:
# dir = "./test/NORMAL/"
# for root,dirs,files in os.walk(dir):
#     for x,file in enumerate(files):
#         curdir =  os.path.join(root,file)
#         img = cv2.imread(curdir,0) 
#         img = cv2.resize(img, (299,299))
#         print(curdir)
#         cv2.imwrite("./test/NORMAL/resized/%d.jpg" % x, img)

In [ ]:
# 1340 (0 idx)	in train/normal 

# 3874		in train/pneumonia 

# 389		in test/pneumonia 

# 233		in test/normal 

In [ ]:
trainlen_norm = list(range(1340))
trainlen_not = list(range(3874))

In [ ]:
class DataGenTRAIN(Sequence):
        def __init__(self, tot_ex, batch_size = 32):
                self.batch_size = batch_size
                self.tot_ex = tot_ex
                self.on_epoch_end()
                self.lwr = 0
                self.upr = batch_size
                self.half = batch_size
                list_ = list(range(0,self.tot_ex))
                np.random.shuffle(list_)
                self.master = list_
        def __getitem__(self, index):
                indices = self.master[self.lwr:self.upr]
                (X,y) =  self.__data_generation(indices = indices )
                self.lwr += self.half
                self.upr +=self.half
                return (X,y)

        def __len__(self):
                return int(np.floor((self.tot_ex) / self.batch_size))
        def on_epoch_end(self):
                self.lwr = 0
                self.upr = self.batch_size
                return

        def __data_generation(self, indices):
                normidxs = random.sample(trainlen_norm, self.batch_size//2) 
                notidxs = random.sample(trainlen_not, self.batch_size//2)
                X= np.array([cv2.imread('./train/NORMAL/resized/%d.jpg' % x,1) for x in normidxs])
                Y = np.ones(self.batch_size//2)
                X_1= np.array([cv2.imread('./train/PNEUMONIA/resized/%d.jpg' % x,1) for x in notidxs])
                Y_1 = np.zeros(self.batch_size//2) 
                indices = list(range(self.batch_size))
                np.random.shuffle(indices)
                X = np.append(X,X_1, axis =0)
                Y = np.append(Y,Y_1,axis=0)
                #print(len(X))
                #print(len(Y))
                X = X[indices] 
                Y = Y[indices]                
                return (X,Y)

In [ ]:
traingenerator = DataGenTRAIN(batch_size = 8, tot_ex = 1340)

In [63]:
inception = InceptionV3(include_top=False, weights='imagenet', classes = 1)
input = Input(shape = (299,299,3))
x = inception(input)
x = Flatten()(x) 
x = Dense(128, activation = 'relu')(x) 
x = Dense(16, activation = 'relu')(x)
output = Dense(1, activation = 'sigmoid')(x) 




In [64]:
model = Model(input = input, output = output)
sgd = optimizers.SGD(lr=0.001, momentum=0.6, decay=1e-6, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer='sgd')

C:\Users\Erik\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [65]:
# # from keras import backend as K
# # K.clear_session()
# del inception

In [75]:
model.fit_generator(generator=traingenerator,workers=1, epochs = 1)

Epoch 1/1
167/167 [==============================] - 20s 121ms/step - loss: 0.0282


In [76]:
# model.save("./79acc_1_8_19.h5")

In [89]:
testlen_norm = list(range(233))
testlen_not = list(range(389))
gt = np.array([])
class DataGenTEST(Sequence):
        def __init__(self, tot_ex, batch_size = 32):
                self.batch_size = batch_size
                self.tot_ex = tot_ex
                self.on_epoch_end()
                self.lwr = 0
                self.upr = batch_size
                self.half = batch_size
                list_ = list(range(0,self.tot_ex))
                np.random.shuffle(list_)
                self.master = list_
        def __getitem__(self, index):
                indices = self.master[self.lwr:self.upr]
                (X,y) =  self.__data_generation(indices = indices )
                self.lwr += self.half
                self.upr +=self.half
                return (X,y)

        def __len__(self):
                return int(np.floor((self.tot_ex) / self.batch_size))
        def on_epoch_end(self):
                self.lwr = 0
                self.upr = self.batch_size
                return

        def __data_generation(self, indices):
                normidxs = random.sample(testlen_norm, self.batch_size//2) 
                notidxs = random.sample(testlen_not, self.batch_size//2)
                X= np.array([cv2.imread('./test/NORMAL/resized/%d.jpg' % x,1) for x in normidxs])
                Y = np.ones(self.batch_size//2)
                X_1= np.array([cv2.imread('./test/PNEUMONIA/resized/%d.jpg' % x,1) for x in notidxs])
                Y_1 = np.zeros(self.batch_size//2) 
                indices = list(range(self.batch_size))
                np.random.shuffle(indices)
                X = np.append(X,X_1, axis =0)
                Y = np.append(Y,Y_1,axis=0)
                #print(len(X))
                #print(len(Y))
                X = X[indices] 
                Y = Y[indices] 
                global gt
                gt = np.append(gt, Y, axis =0)
                return (X,Y)

In [90]:
testgenerator = DataGenTEST(batch_size = 8, tot_ex = 1336)

sample = model.predict_generator(generator=testgenerator,workers=1)
for x in range(len(sample)):
    if sample[x] < .5:
        sample[x] =0
    else:
        sample[x] =1

In [91]:
from sklearn.metrics import accuracy_score
accuracy_score(gt, sample)


0.7305389221556886